# 전처리

라이브러리 불러오기

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import json

데이터 불러오기 및 통합

In [2]:
df_2018 = pd.read_csv('/dshome/ddualab/yohan/finance/data/2018.txt', sep='\t')
df_2019 = pd.read_csv('/dshome/ddualab/yohan/finance/data/2019.txt', sep='\t')
df_2020 = pd.read_csv('/dshome/ddualab/yohan/finance/data/2020.txt', sep='\t')
total_df = pd.concat([df_2018, df_2019, df_2020], axis=0).reset_index(drop = True)

/tmp/ipykernel_2992742/325082414.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2018 = pd.read_csv('/dshome/ddualab/yohan/finance/data/2018.txt', sep='\t')
/tmp/ipykernel_2992742/325082414.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2019 = pd.read_csv('/dshome/ddualab/yohan/finance/data/2019.txt', sep='\t')
/tmp/ipykernel_2992742/325082414.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020 = pd.read_csv('/dshome/ddualab/yohan/finance/data/2020.txt', sep='\t')


불필요 칼럼 삭제 및 NaN 값 확인

In [3]:
del_col = ['고객 이메일', '고객 이름', '구매 상품명', '구매 상품금액', '구매 수향']
usable_col = [value for value in total_df.columns.tolist() if value not in del_col]
total_df = total_df[usable_col]

isna_arr = total_df.isna().sum()
industry_df = total_df[['가맹점아이디', '업종', '업태']]

업종 및 업태의 값이 존재하는 행만 추출

In [4]:
id_arr = total_df.가맹점아이디.value_counts()
nan_id = []
for id in tqdm(id_arr.index):
    isna_sum = industry_df[industry_df['가맹점아이디'] == id].isna().sum()
    if (isna_sum['업종'] > 0) & (isna_sum['업태'] > 0):
        nan_id.append(id)

100%|██████████| 198/198 [00:38<00:00,  5.08it/s]


In [5]:
id_nan_idx = []
for nan_idx in nan_id:
    id_nan_idx.append(total_df[['가맹점아이디']][total_df['가맹점아이디'] == nan_idx].index.tolist())
id_nan_idx = sum(id_nan_idx, [])
eliminate_industry_df = total_df.drop(id_nan_idx).reset_index(drop = True)

국내 위도 경도 정보가 포함된 JSON 형태 파일 불러오기 및 칼럼에 값 추가

In [6]:
# json 파일 불러오기
with open('korea_long_lat.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 출력
coor_df_info = {}
coordinate_df = pd.DataFrame(data)
coordinate_df_t = coordinate_df.transpose()
coor_df_columns = ['name', 'lat', 'long']

coor_df_info['name'] = coordinate_df_t.index
coor_df_info['lat'] = coordinate_df_t.lat.values
coor_df_info['long'] = coordinate_df_t.long.values

coor_df = pd.DataFrame(coor_df_info)

re_name = []
for coor in coor_df.name.values:
    re_name.append(coor.split('/')[-1])

coor_df['re_name'] = re_name

def extract_last_word(region):
    return region.split(' ')[-1] if isinstance(region, str) else None

# '가맹점 지역' 칼럼에서 마지막 단어 추출
eliminate_industry_df['last_word'] = eliminate_industry_df['가맹점 지역'].apply(extract_last_word)


merged_df = pd.merge(eliminate_industry_df, coor_df[['name', 'lat', 'long', 're_name']], 
                      left_on='last_word', right_on='re_name', 
                      how='left')

# merged_df.to_csv('/dshome/ddualab/yohan/finance/yh/eliminate_trash_infor_added_coordinate.csv', index = False)

중간 전처리 된 데이터프레임의 'NaN' 확인 및 제거

In [7]:
merged_df.isna().sum()

거래번호                 0
가맹점아이디               0
업태              152011
업종                6438
사업자구분                0
사업자번호                0
법인등록번호               0
매입사                  0
결제통화                 0
금액                   0
결제사                  0
고객 카드BIN          6468
고객 카드번호 암호화값      6307
등록일                  0
시스템 등록일시             0
결제수단                 0
결제유형                 0
거래상태                 0
가맹점 지역               0
승인 지역                0
연령층                  0
영업시간                 0
last_word            0
name                 0
lat                  0
long                 0
re_name              0
dtype: int64

고객 카드번호 암호화값을 통해 개인을 분류할 수 있으므로, 위 값이 NaN값을 가질 경우 학습에 사용할 수 없는 데이터로 판단하여 삭제

In [8]:
card_pw_nan_idx = merged_df[merged_df[['고객 카드번호 암호화값']].isna().any(axis = 1)].index.tolist()
merged_df = merged_df.drop(card_pw_nan_idx).reset_index(drop = True)
merged_df.isna().sum()

거래번호                 0
가맹점아이디               0
업태              147219
업종                5339
사업자구분                0
사업자번호                0
법인등록번호               0
매입사                  0
결제통화                 0
금액                   0
결제사                  0
고객 카드BIN           161
고객 카드번호 암호화값         0
등록일                  0
시스템 등록일시             0
결제수단                 0
결제유형                 0
거래상태                 0
가맹점 지역               0
승인 지역                0
연령층                  0
영업시간                 0
last_word            0
name                 0
lat                  0
long                 0
re_name              0
dtype: int64

업태 및 업종 중 하나의 값만 포함될 경우 최종 학습형태에 적합하도록 '업태' 칼럼에 모든 값 추가

In [9]:
nan_business_item = merged_df[merged_df[['업태']].isna().any(axis = 1)].업종.value_counts().index
nan_industry_item = merged_df[merged_df[['업종']].isna().any(axis = 1)].업태.value_counts().index

merged_df.loc[merged_df['업종'] == 'Information Products', '업태'] = 'InformationProducts'

for industry_type in nan_industry_item:
    if industry_type == 'Generale-comm':
        merged_df.loc[merged_df['업태'] == industry_type, '업종'] = '도매및소매업'
    elif industry_type == 'ON-LINEMARKETINGANDWHOLESALES':
        merged_df.loc[merged_df['업태'] == industry_type, '업종'] = '도매업'
    elif industry_type == 'TranslationService':
        merged_df.loc[merged_df['업태'] == industry_type, '업종'] = '서비스업'
    elif industry_type == 'Cosmetics':
        merged_df.loc[merged_df['업태'] == industry_type, '업종'] = '도매및소매업'
    elif industry_type == '온라인번역':
        merged_df.loc[merged_df['업태'] == industry_type, '업종'] = '서비스업'



원화 일치를 위한 각 환율에 적합하도록 변경

In [10]:
merged_df["원화환산금액"] = merged_df["금액"] 
merged_df.loc[merged_df["결제통화"] == "JPY","원화환산금액"] = merged_df[merged_df["결제통화"] == "JPY"]["원화환산금액"] * 9.20
merged_df.loc[merged_df["결제통화"] == "USD","원화환산금액"] = merged_df[merged_df["결제통화"] == "USD"]["원화환산금액"] * 1379
merged_df['업태'] = merged_df['업태'].str.replace(' ', '')

업태 통일화를 위한 수기분류 작업

In [11]:
도소매=['도소매업외','도매및소매업','도소매','도소매,소매','도매및','도매및소매업,금융및보험업','도매및소매업외','도소매업,서비스업','도소매업','도매및상품중개업','도소매업,서비스업','도매및기타중개업','도매및소매업,기타서비스업','소매','도매및소매업,서비스','도소매외','도매및소매업,서비스업','도소매,서비스','소매업']
도소매 = list(set(도소매))

서비스=['서비스,소매','서비스업,정보통신업','서비스업','서비스,도소매','서비스','서비스업외','서비스,금융업,서비스','서비스,금융업','서비스.금융업','서비스,금융업,서비스','서비스업','서비스,제조업,소매업,도매업','기술서비스업','서비스,서비스업']
서비스 = list(set(서비스))

Information_Products=['InformationProducts']
Information_Products = list(set(Information_Products))

통신업=['통신업,도소매,도매','정보통신업']
통신업 = list(set(통신업))

General_e_comm=['Generale-comm','소프트웨어개발및공급','소프트웨어자문,개발및공급,전자상거래업','전자상거래']
General_e_comm = list(set(General_e_comm))

Online_marketing=['ON-LINEMARKETINGANDWHOLESALES','온라인쇼핑몰']
Online_marketing = list(set(Online_marketing))

음식=['음식','음식및숙박업','음식점','음식점업','유흥주점']
음식 = list(set(음식))

Translation_service=['TranslationService','온라인번역']
Translation_service = list(set(Translation_service))

Cosmetics=['Cosmetics']
Cosmetics = list(set(Cosmetics))

금융업=['금융업']
금융업 = list(set(금융업))

의류=['의류,행사대행,기타대리']
의류 = list(set(의류))

모바일=['모바일결제기']
모바일 = list(set(모바일))

pranchise_idx_key = ['도소매', '서비스', 'Information_Products', '통신업', 'General_e_comm', 'Online_marketing', '음식', 'Translation_service', 'Cosmetics', '금융업', '의류', '모바일']

pranchise_idx = [도소매, 서비스, Information_Products, 통신업, General_e_comm, Online_marketing, 음식, Translation_service, Cosmetics, 금융업, 의류, 모바일]

수기로 분류된 각 형태에 맞도록 변환

In [12]:
new_total_pranchise = []
for df_pranchise_value in tqdm(merged_df.업태.values):
# for total_idx, df_pranchise_value in enumerate(total_df.업태.values, 0):

    for idx, all_pranchise_value in enumerate(pranchise_idx, 0):
        
        if df_pranchise_value.strip() in all_pranchise_value:
            # print(df_pranchise_value)
            new_total_pranchise.append(pranchise_idx_key[idx])
            # print(pranchise_idx_key[idx])
        # else:
            # print(df_pranchise_value)
merged_df['new_업태'] = new_total_pranchise

100%|██████████| 2486947/2486947 [00:06<00:00, 388590.79it/s]


새롭게 할당된 업태를 가지는 값 확인 (12개로 축소화 및 일치화)

In [13]:
merged_df['new_업태'].unique()

array(['서비스', '통신업', '도소매', '의류', '모바일', '음식', '금융업', 'General_e_comm',
       'Online_marketing', 'Information_Products', 'Translation_service',
       'Cosmetics'], dtype=object)

취소가 존재하는 정보 추출 및 데이터 저장 (이상 및 정상 데이터)
- 이상치 데이터: fds
- 정상 데이터: norm

In [14]:
merged_df.isna().sum()

거래번호               0
가맹점아이디             0
업태                 0
업종              5278
사업자구분              0
사업자번호              0
법인등록번호             0
매입사                0
결제통화               0
금액                 0
결제사                0
고객 카드BIN         161
고객 카드번호 암호화값       0
등록일                0
시스템 등록일시           0
결제수단               0
결제유형               0
거래상태               0
가맹점 지역             0
승인 지역              0
연령층                0
영업시간               0
last_word          0
name               0
lat                0
long               0
re_name            0
원화환산금액             0
new_업태             0
dtype: int64

승인취소를 예측하기 위한 데이터프레임을 생성할 경우
- issue_df1 사용

승인 및 매입취소 데이터를 모두 사용하기 위한 데이터프레임을 생성할 경우
- issue_df1, issue_df2 모두 사용 후 pd.concat을 통해 데이터프레임 합

In [15]:
total_df_c = merged_df.copy()[['고객 카드번호 암호화값', '거래상태']]
# issue_df1 = total_df_c[total_df_c['거래상태'] == '승인취소']
issue_df2 = total_df_c[total_df_c['거래상태'] == '매입취소']
# issue_df = pd.concat([issue_df1, issue_df2], axis = 0).reset_index(drop = True)
# issue_df_ = pd.concat([issue_df1, issue_df2], axis = 0)

# cancel_idx = issue_df_.index
cancel_idx = issue_df2.index

fds_index = []
for i in tqdm(issue_df2['고객 카드번호 암호화값'].values):
    true_df = total_df_c[total_df_c['고객 카드번호 암호화값'] == i]
    if len(true_df.index) > 0:
        fds_index.append(true_df.index[0])
    else:
        continue

total_fds_idx = np.concatenate([np.array(fds_index, dtype=np.int64), np.array(cancel_idx, dtype = np.int64)])

fds_df =merged_df.iloc[fds_index, :].reset_index(drop = True)
# fds_df.to_csv('fds_df_total.csv', index = False)

norm_df = merged_df.drop(total_fds_idx).reset_index(drop = True)
# norm_df.to_csv('norm_df_total.csv', index = False)

100%|██████████| 5843/5843 [13:32<00:00,  7.19it/s]


추출된 정상 및 이상치 데이터 프레임에 대해 라벨링 및 통합 후 저장
- 정상: 0
- 비정상: 1

In [65]:
fds_df['label'] = 1
norm_df['label'] = 0

# 두 데이터프레임을 합쳐서 last_total_df 생성
last_total_df = pd.concat([fds_df, norm_df], axis=0).reset_index(drop=True)
last_total_df.to_csv('last_total.csv', index = False)

지정한 새로운 업태에 해당하는 행을 추출하여 업태 별 데이터프레임 저장
- 업태 별 데이터 저장이 필요한 경우에만 실행

In [ ]:
for pranchise_name in pranchise_idx_key:
    last_total_df[last_total_df['new_업태'] == pranchise_name].reset_index(drop = True).to_csv('{}.csv'.format(pranchise_name), index = False)

각 데이터를 불러온 뒤 이상치, 정상 라벨 수 확인

In [64]:
for pranchise_name in pranchise_idx_key:
    current_df = pd.read_csv('{}.csv'.format(pranchise_name))
    print(current_df['label'].value_counts())

/tmp/ipykernel_2989375/1408069271.py:2: DtypeWarning: Columns (7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  current_df = pd.read_csv('{}.csv'.format(pranchise_name))


1    1362874
0       3453
Name: label, dtype: int64
1    874701
0      2339
Name: label, dtype: int64
1    147219
Name: label, dtype: int64
1    84266
0       29
Name: label, dtype: int64
1    3195
0      21
Name: label, dtype: int64
1    2033
Name: label, dtype: int64
1    697
0      1
Name: label, dtype: int64
1    250
Name: label, dtype: int64
1    53
Name: label, dtype: int64
1    38
Name: label, dtype: int64
1    10
Name: label, dtype: int64
1    4
Name: label, dtype: int64
